In [ ]:
#### Regression 

import numpy as np
import pandas as pd

file_path = '..\Astro-AI-Chatbot\linear_regression\Files_for_Priming_Sector_Comps\output_file_with_growth10.xlsx'
df = pd.read_excel(file_path)
print(df)

target_column = 'Target'
feature_columns = ['feature1', 'feature2']

x = df[feature_columns].values
y = df[target_column].values

def gradient_descent(x, y, learning_rate=0.01, num_iterations=1000):
    m, n = x.shape
    theta = np.zeros(n)
    price_history = []

    for i in range(num_iterations):
        predictions = np.dot(x, theta)
        error = predictions - y
        gradient = (1/m) * np.dot(x.T, error)
        theta -= learning_rate * gradient

        price = (1/(2*m)) * np.sum(error ** 2)
        price_history.append(price)

    return theta, price_history

# Usage
optimal_theta, price_history = gradient_descent(x, y)
print("Optimal Theta:", optimal_theta)
print("Future Price:", price_history[-1])

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import re

# Function to convert a string percentage to a float
def convert_percentage_to_float(perc):
    try:
        return float(perc.strip('%')) / 100
    except:
        return np.nan

# Read the datasets
growth_df = pd.read_excel('output_file_with_growth10.xlsx')
returns_df = pd.read_excel('returntable.xlsx')

# Extract YOY Growth columns and their corresponding years
yoy_growth_cols = [col for col in growth_df.columns if re.match(r'\d{4} - \d{4} YOY Growth', col)]
growth_years = [re.findall(r'(\d{4}) - \d{4} YOY Growth', col)[0] for col in yoy_growth_cols]

# Convert year in returns_df to string for alignment
returns_df['Year'] = returns_df['Year'].astype(str)

# Initialize an empty DataFrame for merged data
merged_data = pd.DataFrame()

# Iterate through YOY Growth columns and corresponding years to merge data
for growth_col, year in zip(yoy_growth_cols, growth_years):
    growth_data = growth_df[['Unnamed: 0', growth_col]]
    growth_data = growth_data.rename(columns={'Unnamed: 0': 'Category', growth_col: 'YOY Growth'})
    growth_data['Year'] = year
    merged_year_data = pd.merge(growth_data, returns_df, on='Year')
    merged_data = pd.concat([merged_data, merged_year_data])

# Convert YOY Growth percentage to numeric values and drop NaNs
merged_data['YOY Growth'] = merged_data['YOY Growth'].apply(convert_percentage_to_float)
cleaned_data = merged_data.dropna(subset=['YOY Growth', 'S&P 500 (includes dividends)'])

# Prepare the data for regression
X = cleaned_data[['YOY Growth']]  # Predictor variable
y = cleaned_data['S&P 500 (includes dividends)']  # Response variable

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Coefficients and Intercept
coef = model.coef_[0]
intercept = model.intercept_

# Output the results
coef, intercept
